# 2025-02-07 22:52:10	0.7293877653 1128411

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMClassifier

# 1. 학습 데이터 불러오기 및 전처리
train_df = pd.read_csv('train.csv')

# ID 관련 컬럼 제거 (ID 컬럼은 모델 학습에 사용하지 않음)
for col in ['id', 'ID', 'RecordID', 'patient_id']:
    if col in train_df.columns:
        train_df = train_df.drop(col, axis=1)

# 타깃과 피처 분리 (타깃 컬럼명이 '임신 성공 여부'라고 가정)
target = '임신 성공 여부'
X_train = train_df.drop(target, axis=1)
y_train = train_df[target]

# 범주형, 수치형 변수 지정 (아래 목록은 예시이므로 실제 데이터에 맞게 수정하세요)
categorical_columns = [
    '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형',  # "특정 시술 유형"은 문자열이므로 반드시 범주형에 포함!
    '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부',
    '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
    '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
    '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
    '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인',
    '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
    '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
    '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유',
    '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수',
    '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수',
    'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처',
    '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부',
    '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부',
    'PGD 시술 여부', 'PGS 시술 여부'
]
numerical_columns = [col for col in X_train.columns if col not in categorical_columns]

# 전처리 파이프라인 구성 (수치형: StandardScaler, 범주형: OneHotEncoder)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

# 2. 모델 정의 및 파이프라인 구성
# 최적의 하이퍼파라미터 (예: n_estimators=100, learning_rate=0.01, max_depth=5)를 사용
classifier = LGBMClassifier(
    n_estimators=100,
    learning_rate=0.01,
    max_depth=5,
    random_state=42
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])

# 학습 데이터 전체로 모델 학습
pipeline.fit(X_train, y_train)
print("모델 학습 완료.")

# 3. 테스트 데이터 불러오기 및 예측
# 테스트 데이터에 ID 컬럼이 포함되어 있다고 가정 (예: "ID" 컬럼)
test_df = pd.read_csv('test.csv')

if 'ID' in test_df.columns:
    test_ids = test_df['ID']
    X_test = test_df.drop('ID', axis=1)
elif 'id' in test_df.columns:
    test_ids = test_df['id']
    X_test = test_df.drop('id', axis=1)
else:
    # ID 컬럼이 없으면 인덱스를 문자열로 변환하여 사용
    test_ids = test_df.index.astype(str)
    X_test = test_df.copy()

# 예측 확률 (양성 클래스에 대한 확률)
probabilities = pipeline.predict_proba(X_test)[:, 1]

# 4. 결과 DataFrame 생성 및 CSV 저장
result_df = pd.DataFrame({
    'ID': test_ids,
    'probability': probabilities
})

result_df.to_csv('Result.csv', index=False)
print("Result.csv 파일이 생성되었습니다.")


c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032407 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 913
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 190
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Result.csv 파일이 생성되었습니다.
